In [2]:
import pandas as pd
import numpy as np
import mcdm

In [3]:
shopper_df = pd.read_csv('/Users/nitanshjain/Documents/Projects/Shopper_Intent_Prediction/shopper-intent-prediction/topsis_data/topsis_short.csv')
print(shopper_df.shape)
print(shopper_df.columns)
shopper_df.head()

(3600, 16)
Index(['length_text', 'samples', 'models', 'percentiles', 'mi_accuracy',
       'mi_f1_score', 'mi_auc', 'mrmr_accuracy', 'mrmr_f1_score', 'mrmr_auc',
       'mi_mrmr_accuracy', 'mi_mrmr_f1_score', 'mi_mrmr_auc', 'pca_accuracy',
       'pca_f1_score', 'pca_auc'],
      dtype='object')


,length_text,samples,models,percentiles,mi_accuracy,mi_f1_score,mi_auc,mrmr_accuracy,mrmr_f1_score,mrmr_auc,mi_mrmr_accuracy,mi_mrmr_f1_score,mi_mrmr_auc,pca_accuracy,pca_f1_score,pca_auc
0,8,subsample_5_v2.csv,ft_transformer,10,0.7137,0.6516,0.7512,0.6785,0.6272,0.7372,0.6918,0.6256,0.7413,0.6928,0.6275,0.7396
1,8,subsample_9_v2.csv,ft_transformer,10,0.7137,0.6596,0.7584,0.6891,0.6465,0.7507,0.7157,0.6532,0.7420,0.7004,0.6334,0.7517
2,8,subsample_3_v2.csv,ft_transformer,10,0.7017,0.6347,0.7416,0.7054,0.6457,0.7446,0.6931,0.6254,0.7414,0.6772,0.6346,0.7313
3,8,subsample_6_v2.csv,ft_transformer,10,0.7004,0.6328,0.7537,0.7067,0.6453,0.7479,0.6894,0.6211,0.7548,0.7014,0.6386,0.7520
4,8,subsample_7_v2.csv,ft_transformer,10,0.7090,0.6416,0.7523,0.6921,0.6769,0.7460,0.6858,0.6756,0.7324,0.6954,0.6721,0.7495


In [4]:
print(shopper_df.length_text.value_counts())
print(shopper_df.models.value_counts())
print(shopper_df.percentiles.value_counts())

length_text
8     360
9     360
10    360
11    360
12    360
13    360
14    360
15    360
16    360
17    360
Name: count, dtype: int64
models
ft_transformer    600
lr                600
rfc               600
xgbc              600
lgbm              600
svm               600
Name: count, dtype: int64
percentiles
10    600
20    600
30    600
50    600
75    600
90    600
Name: count, dtype: int64


In [5]:
# getting all columns that have a float64 type
shopper_df_float = shopper_df.select_dtypes(include=['float64']).columns
shopper_df_float

Index(['mi_accuracy', 'mi_f1_score', 'mi_auc', 'mrmr_accuracy',
       'mrmr_f1_score', 'mrmr_auc', 'mi_mrmr_accuracy', 'mi_mrmr_f1_score',
       'mi_mrmr_auc', 'pca_accuracy', 'pca_f1_score', 'pca_auc'],
      dtype='object')

In [6]:
def mean_models(length, df=shopper_df): 
    percentiles_list = [10, 20, 30, 50, 75, 90]
    models_list = ['ft_transformer', 'lr', 'rfc', 'xgbc', 'lgbm', 'svm']
    all_df = list()
    
    for j in percentiles_list:
        means = list()
        for i in models_list:
            model = df[(df.percentiles == j) & (df.models == i) & (df.length_text == length)]
            # print(model)
            mean_val = list(model[shopper_df_float].mean())
            means.append(mean_val)
        means_df = pd.DataFrame(means, columns=shopper_df_float).reset_index()
        means_df['models_name'] = pd.Series(['ft_transformer', 'lr', 'rfc', 'xgbc', 'lgbm', 'svm'])
        means_df['percentiles_name'] = pd.Series([j] * 6)
        # print(means_df.head())
        all_df.append(means_df)
        
    ranks_model_per_percentile = list()
    for i in all_df:
        i.drop(['index', 'models_name', 'percentiles_name'], axis=1, inplace=True)
        ranks_model_per_percentile.append(mcdm.rank(i.to_numpy(), s_method='TOPSIS', alt_names=models_list))
        
    best_models = list()
    for val in ranks_model_per_percentile:
        best_models_per_percentile = list()
        # print(val)
        for i in val:
                best_models_per_percentile.append(i[0])
                
        best_models.append(best_models_per_percentile)
        
    models_df = pd.DataFrame(best_models, columns=['model1', 'model2', 'model3', 'model4', 'model5', 'model6'])
    models_df['percentiles'] = pd.Series([10, 20, 30, 50, 75, 90])
    return models_df

best_models_8 = mean_models(8)
best_models_9 = mean_models(9)
best_models_10 = mean_models(10)
best_models_11 = mean_models(11)
best_models_12 = mean_models(12)
best_models_13 = mean_models(13)
best_models_14 = mean_models(14)
best_models_15 = mean_models(15)
best_models_16 = mean_models(16)
best_models_17 = mean_models(17)

print(best_models_8)
print(best_models_9)
print(best_models_10)
print(best_models_11)
print(best_models_12)
print(best_models_13)
print(best_models_14)
print(best_models_15)
print(best_models_16)
print(best_models_17)

           model1 model2 model3 model4 model5 model6  percentiles
0  ft_transformer    svm   lgbm   xgbc    rfc     lr           10
1  ft_transformer    svm   lgbm     lr   xgbc    rfc           20
2  ft_transformer    svm   lgbm   xgbc     lr    rfc           30
3  ft_transformer     lr    svm   lgbm   xgbc    rfc           50
4  ft_transformer     lr    svm   lgbm    rfc   xgbc           75
5  ft_transformer     lr   lgbm    svm   xgbc    rfc           90
           model1 model2 model3 model4 model5 model6  percentiles
0  ft_transformer    svm   xgbc   lgbm    rfc     lr           10
1  ft_transformer    svm   lgbm    rfc   xgbc     lr           20
2  ft_transformer    svm   xgbc   lgbm     lr    rfc           30
3  ft_transformer    svm   lgbm     lr   xgbc    rfc           50
4  ft_transformer   lgbm    svm   xgbc     lr    rfc           75
5  ft_transformer    svm     lr   lgbm   xgbc    rfc           90
           model1 model2 model3 model4 model5 model6  percentiles
0  ft_tran

In [22]:
def mean_percentiles(length, df=shopper_df): 
    percentiles_list = [10, 20, 30, 50, 75, 90]
    models_list = ['ft_transformer', 'lr', 'rfc', 'xgbc', 'lgbm', 'svm']
    all_df = list()
    for i in models_list:
        means = list()
        for j in percentiles_list:
            model = df[(df.percentiles == j) & (df.models == i) & (df.length_text == length)]
            mean_val = list(model[shopper_df_float].mean())
            means.append(mean_val)
        means_df = pd.DataFrame(means, columns=shopper_df_float).reset_index()
        means_df['percentiles_name'] = pd.Series([10, 20, 30, 50, 75, 90])
        means_df['models_name'] = pd.Series([model] * 6)
        all_df.append(means_df)
        
    ranks_percentile_per_model = list()
    for i in all_df:
        i.drop(['index', 'models_name', 'percentiles_name'], axis=1, inplace=True)
        ranks_percentile_per_model.append(mcdm.rank(i.to_numpy(), s_method='TOPSIS', alt_names=percentiles_list, is_benefit_x=[True]*len(shopper_df_float)))
        
    best_percentiles = list()
    for val in ranks_percentile_per_model:
        best_percentile_per_model = list()
        # print(val)
        for i in val:
            best_percentile_per_model.append(i[0])
        best_percentiles.append(best_percentile_per_model)
        
    models_df = pd.DataFrame(best_percentiles, columns=['per1', 'per2', 'per3', 'per4', 'per5', 'per6'])
    models_df['models'] = pd.Series(['ft_transformer', 'lr', 'rfc', 'xgbc', 'lgbm', 'svm'])
    return models_df

best_percentiles_8 = mean_percentiles(8)
best_percentiles_9 = mean_percentiles(9)
best_percentiles_10 = mean_percentiles(10)
best_percentiles_11 = mean_percentiles(11)
best_percentiles_12 = mean_percentiles(12)
best_percentiles_13 = mean_percentiles(13)
best_percentiles_14 = mean_percentiles(14)
best_percentiles_15 = mean_percentiles(15)
best_percentiles_16 = mean_percentiles(16)
best_percentiles_17 = mean_percentiles(17)

print(best_percentiles_8)
print(best_percentiles_9)
print(best_percentiles_10)
print(best_percentiles_11)
print(best_percentiles_12)
print(best_percentiles_13)
print(best_percentiles_14)
print(best_percentiles_15)
print(best_percentiles_16)
print(best_percentiles_17)

   per1  per2  per3  per4  per5  per6          models
0    30    75    20    50    90    10  ft_transformer
1    75    90    50    30    20    10              lr
2    50    75    90    20    30    10             rfc
3    50    30    90    20    75    10            xgbc
4    50    90    75    30    20    10            lgbm
5    50    30    20    90    75    10             svm
   per1  per2  per3  per4  per5  per6          models
0    50    30    90    75    20    10  ft_transformer
1    50    75    90    30    20    10              lr
2    75    90    50    30    20    10             rfc
3    75    50    90    30    20    10            xgbc
4    75    50    90    30    20    10            lgbm
5    30    75    50    20    90    10             svm
   per1  per2  per3  per4  per5  per6          models
0    20    75    90    30    50    10  ft_transformer
1    75    90    50    30    20    10              lr
2    75    90    50    30    20    10             rfc
3    75    50    30    90   

In [8]:
def getting_mean(length, df=shopper_df):
    percentiles_list = [10, 20, 30, 50, 75, 90]
    models_list = ['ft_transformer', 'lr', 'rfc', 'xgbc', 'lgbm', 'svm']
    all_df = list()
    for j in percentiles_list:
        means = list()
        
        for i in models_list:
            model = df[(df.percentiles == j) & (df.models == i) & (df.length_text == length)]
            mean_val = list(model[shopper_df_float].mean())
            means.append(mean_val)
        means_df = pd.DataFrame(means, columns=shopper_df_float).reset_index()
        means_df['models_name'] = pd.Series(['ft_transformer', 'lr', 'rfc', 'xgbc', 'lgbm', 'svm'])
        means_df['percentiles_name'] = pd.Series([j] * 6)
        all_df.append(means_df)
    all_df = pd.concat(all_df)
    all_df.drop(['index', 'models_name', 'percentiles_name'], axis=1, inplace=True)
    return all_df

feature_reduction_df_8 = getting_mean(8)
feature_reduction_df_9 = getting_mean(9)
feature_reduction_df_10 = getting_mean(10)
feature_reduction_df_11 = getting_mean(11)
feature_reduction_df_12 = getting_mean(12)
feature_reduction_df_13 = getting_mean(13)
feature_reduction_df_14 = getting_mean(14)
feature_reduction_df_15 = getting_mean(15)
feature_reduction_df_16 = getting_mean(16)
feature_reduction_df_17 = getting_mean(17)

print(feature_reduction_df_8)
print(feature_reduction_df_9)
print(feature_reduction_df_10)
print(feature_reduction_df_11)
print(feature_reduction_df_12)
print(feature_reduction_df_13)
print(feature_reduction_df_14)
print(feature_reduction_df_15)
print(feature_reduction_df_16)
print(feature_reduction_df_17)

   mi_accuracy  mi_f1_score    mi_auc  mrmr_accuracy  mrmr_f1_score  mrmr_auc  \
0     0.706470     0.644810  0.752120       0.700700       0.647460  0.748670   
1     0.705441     0.657071  0.705441       0.706171       0.651929  0.706171   
2     0.707896     0.654670  0.707896       0.709721       0.652345  0.709721   
3     0.711546     0.655753  0.711546       0.711745       0.652491  0.711745   
4     0.711646     0.655358  0.711646       0.711878       0.652182  0.711878   
5     0.713603     0.656924  0.713603       0.712276       0.652560  0.712276   
0     0.710310     0.653510  0.753850       0.710020       0.648090  0.749740   
1     0.709754     0.660716  0.709754       0.709522       0.655777  0.709522   
2     0.712243     0.652631  0.712243       0.712177       0.653026  0.712177   
3     0.713404     0.656905  0.713404       0.712674       0.656461  0.712674   
4     0.713603     0.659719  0.713603       0.712807       0.657417  0.712807   
5     0.714831     0.657855 

In [21]:
print("============================================================ 8= ===========================================================")
feature_reduction_df_8_f1 = feature_reduction_df_8[['mi_f1_score', 'mrmr_f1_score', 'mi_mrmr_f1_score', 'pca_f1_score']].reset_index(drop=True).T
feature_reduction_df_8_auc = feature_reduction_df_8[['mi_auc', 'mrmr_auc', 'mi_mrmr_auc', 'pca_auc']].reset_index(drop=True).T
feature_reduction_df_8_accuracy = feature_reduction_df_8[['mi_accuracy', 'mrmr_accuracy', 'mi_mrmr_accuracy', 'pca_accuracy']].reset_index(drop=True).T
print(mcdm.rank(feature_reduction_df_8_f1.to_numpy(), s_method='TOPSIS', alt_names=['mi_f1_score', 'mrmr_f1_score', 'mi_mrmr_f1_score', 'pca_f1_score']))
print(mcdm.rank(feature_reduction_df_8_auc.to_numpy(), s_method='TOPSIS', alt_names=['mi_auc', 'mrmr_auc', 'mi_mrmr_auc', 'pca_auc']))
print(mcdm.rank(feature_reduction_df_8_accuracy.to_numpy(), s_method='TOPSIS', alt_names=['mi_accuracy', 'mrmr_accuracy', 'mi_mrmr_accuracy', 'pca_accuracy']))

print("============================================================ 9= ===========================================================")
feature_reduction_df_9_f1 = feature_reduction_df_9[['mi_f1_score', 'mrmr_f1_score', 'mi_mrmr_f1_score', 'pca_f1_score']].reset_index(drop=True).T
feature_reduction_df_9_auc = feature_reduction_df_9[['mi_auc', 'mrmr_auc', 'mi_mrmr_auc', 'pca_auc']].reset_index(drop=True).T
feature_reduction_df_9_accuracy = feature_reduction_df_9[['mi_accuracy', 'mrmr_accuracy', 'mi_mrmr_accuracy', 'pca_accuracy']].reset_index(drop=True).T
print(mcdm.rank(feature_reduction_df_9_f1.to_numpy(), s_method='TOPSIS', alt_names=['mi_f1_score', 'mrmr_f1_score', 'mi_mrmr_f1_score', 'pca_f1_score']))
print(mcdm.rank(feature_reduction_df_9_auc.to_numpy(), s_method='TOPSIS', alt_names=['mi_auc', 'mrmr_auc', 'mi_mrmr_auc', 'pca_auc']))
print(mcdm.rank(feature_reduction_df_9_accuracy.to_numpy(), s_method='TOPSIS', alt_names=['mi_accuracy', 'mrmr_accuracy', 'mi_mrmr_accuracy', 'pca_accuracy']))

print("============================================================ 10 ============================================================")
feature_reduction_df_10_f1 = feature_reduction_df_10[['mi_f1_score', 'mrmr_f1_score', 'mi_mrmr_f1_score', 'pca_f1_score']].reset_index(drop=True).T
feature_reduction_df_10_auc = feature_reduction_df_10[['mi_auc', 'mrmr_auc', 'mi_mrmr_auc', 'pca_auc']].reset_index(drop=True).T
feature_reduction_df_10_accuracy = feature_reduction_df_10[['mi_accuracy', 'mrmr_accuracy', 'mi_mrmr_accuracy', 'pca_accuracy']].reset_index(drop=True).T
print(mcdm.rank(feature_reduction_df_10_f1.to_numpy(), s_method='TOPSIS', alt_names=['mi_f1_score', 'mrmr_f1_score', 'mi_mrmr_f1_score', 'pca_f1_score']))
print(mcdm.rank(feature_reduction_df_10_auc.to_numpy(), s_method='TOPSIS', alt_names=['mi_auc', 'mrmr_auc', 'mi_mrmr_auc', 'pca_auc']))
print(mcdm.rank(feature_reduction_df_10_accuracy.to_numpy(), s_method='TOPSIS', alt_names=['mi_accuracy', 'mrmr_accuracy', 'mi_mrmr_accuracy', 'pca_accuracy']))

print("============================================================ 11 ============================================================")
feature_reduction_df_11_f1 = feature_reduction_df_11[['mi_f1_score', 'mrmr_f1_score', 'mi_mrmr_f1_score', 'pca_f1_score']].reset_index(drop=True).T
feature_reduction_df_11_auc = feature_reduction_df_11[['mi_auc', 'mrmr_auc', 'mi_mrmr_auc', 'pca_auc']].reset_index(drop=True).T
feature_reduction_df_11_accuracy = feature_reduction_df_11[['mi_accuracy', 'mrmr_accuracy', 'mi_mrmr_accuracy', 'pca_accuracy']].reset_index(drop=True).T
print(mcdm.rank(feature_reduction_df_11_f1.to_numpy(), s_method='TOPSIS', alt_names=['mi_f1_score', 'mrmr_f1_score', 'mi_mrmr_f1_score', 'pca_f1_score']))
print(mcdm.rank(feature_reduction_df_11_auc.to_numpy(), s_method='TOPSIS', alt_names=['mi_auc', 'mrmr_auc', 'mi_mrmr_auc', 'pca_auc']))
print(mcdm.rank(feature_reduction_df_11_accuracy.to_numpy(), s_method='TOPSIS', alt_names=['mi_accuracy', 'mrmr_accuracy', 'mi_mrmr_accuracy', 'pca_accuracy']))

print("============================================================ 12 ============================================================")
feature_reduction_df_12_f1 = feature_reduction_df_12[['mi_f1_score', 'mrmr_f1_score', 'mi_mrmr_f1_score', 'pca_f1_score']].reset_index(drop=True).T
feature_reduction_df_12_auc = feature_reduction_df_12[['mi_auc', 'mrmr_auc', 'mi_mrmr_auc', 'pca_auc']].reset_index(drop=True).T
feature_reduction_df_12_accuracy = feature_reduction_df_12[['mi_accuracy', 'mrmr_accuracy', 'mi_mrmr_accuracy', 'pca_accuracy']].reset_index(drop=True).T
print(mcdm.rank(feature_reduction_df_12_f1.to_numpy(), s_method='TOPSIS', alt_names=['mi_f1_score', 'mrmr_f1_score', 'mi_mrmr_f1_score', 'pca_f1_score']))
print(mcdm.rank(feature_reduction_df_12_auc.to_numpy(), s_method='TOPSIS', alt_names=['mi_auc', 'mrmr_auc', 'mi_mrmr_auc', 'pca_auc']))
print(mcdm.rank(feature_reduction_df_12_accuracy.to_numpy(), s_method='TOPSIS', alt_names=['mi_accuracy', 'mrmr_accuracy', 'mi_mrmr_accuracy', 'pca_accuracy']))

print("============================================================ 13 ============================================================")
feature_reduction_df_13_f1 = feature_reduction_df_13[['mi_f1_score', 'mrmr_f1_score', 'mi_mrmr_f1_score', 'pca_f1_score']].reset_index(drop=True).T
feature_reduction_df_13_auc = feature_reduction_df_13[['mi_auc', 'mrmr_auc', 'mi_mrmr_auc', 'pca_auc']].reset_index(drop=True).T
feature_reduction_df_13_accuracy = feature_reduction_df_13[['mi_accuracy', 'mrmr_accuracy', 'mi_mrmr_accuracy', 'pca_accuracy']].reset_index(drop=True).T
print(mcdm.rank(feature_reduction_df_13_f1.to_numpy(), s_method='TOPSIS', alt_names=['mi_f1_score', 'mrmr_f1_score', 'mi_mrmr_f1_score', 'pca_f1_score']))
print(mcdm.rank(feature_reduction_df_13_auc.to_numpy(), s_method='TOPSIS', alt_names=['mi_auc', 'mrmr_auc', 'mi_mrmr_auc', 'pca_auc']))
print(mcdm.rank(feature_reduction_df_13_accuracy.to_numpy(), s_method='TOPSIS', alt_names=['mi_accuracy', 'mrmr_accuracy', 'mi_mrmr_accuracy', 'pca_accuracy']))

print("============================================================ 14 ============================================================")
feature_reduction_df_14_f1 = feature_reduction_df_14[['mi_f1_score', 'mrmr_f1_score', 'mi_mrmr_f1_score', 'pca_f1_score']].reset_index(drop=True).T
feature_reduction_df_14_auc = feature_reduction_df_14[['mi_auc', 'mrmr_auc', 'mi_mrmr_auc', 'pca_auc']].reset_index(drop=True).T
feature_reduction_df_14_accuracy = feature_reduction_df_14[['mi_accuracy', 'mrmr_accuracy', 'mi_mrmr_accuracy', 'pca_accuracy']].reset_index(drop=True).T
print(mcdm.rank(feature_reduction_df_14_f1.to_numpy(), s_method='TOPSIS', alt_names=['mi_f1_score', 'mrmr_f1_score', 'mi_mrmr_f1_score', 'pca_f1_score']))
print(mcdm.rank(feature_reduction_df_14_auc.to_numpy(), s_method='TOPSIS', alt_names=['mi_auc', 'mrmr_auc', 'mi_mrmr_auc', 'pca_auc']))
print(mcdm.rank(feature_reduction_df_14_accuracy.to_numpy(), s_method='TOPSIS', alt_names=['mi_accuracy', 'mrmr_accuracy', 'mi_mrmr_accuracy', 'pca_accuracy']))

print("============================================================ 15 ============================================================")
feature_reduction_df_15_f1 = feature_reduction_df_15[['mi_f1_score', 'mrmr_f1_score', 'mi_mrmr_f1_score', 'pca_f1_score']].reset_index(drop=True).T
feature_reduction_df_15_auc = feature_reduction_df_15[['mi_auc', 'mrmr_auc', 'mi_mrmr_auc', 'pca_auc']].reset_index(drop=True).T
feature_reduction_df_15_accuracy = feature_reduction_df_15[['mi_accuracy', 'mrmr_accuracy', 'mi_mrmr_accuracy', 'pca_accuracy']].reset_index(drop=True).T
print(mcdm.rank(feature_reduction_df_15_f1.to_numpy(), s_method='TOPSIS', alt_names=['mi_f1_score', 'mrmr_f1_score', 'mi_mrmr_f1_score', 'pca_f1_score']))
print(mcdm.rank(feature_reduction_df_15_auc.to_numpy(), s_method='TOPSIS', alt_names=['mi_auc', 'mrmr_auc', 'mi_mrmr_auc', 'pca_auc']))
print(mcdm.rank(feature_reduction_df_15_accuracy.to_numpy(), s_method='TOPSIS', alt_names=['mi_accuracy', 'mrmr_accuracy', 'mi_mrmr_accuracy', 'pca_accuracy']))

print("============================================================ 16 ============================================================")
feature_reduction_df_16_f1 = feature_reduction_df_16[['mi_f1_score', 'mrmr_f1_score', 'mi_mrmr_f1_score', 'pca_f1_score']].reset_index(drop=True).T
feature_reduction_df_16_auc = feature_reduction_df_16[['mi_auc', 'mrmr_auc', 'mi_mrmr_auc', 'pca_auc']].reset_index(drop=True).T
feature_reduction_df_16_accuracy = feature_reduction_df_16[['mi_accuracy', 'mrmr_accuracy', 'mi_mrmr_accuracy', 'pca_accuracy']].reset_index(drop=True).T
print(mcdm.rank(feature_reduction_df_16_f1.to_numpy(), s_method='TOPSIS', alt_names=['mi_f1_score', 'mrmr_f1_score', 'mi_mrmr_f1_score', 'pca_f1_score']))
print(mcdm.rank(feature_reduction_df_16_auc.to_numpy(), s_method='TOPSIS', alt_names=['mi_auc', 'mrmr_auc', 'mi_mrmr_auc', 'pca_auc']))
print(mcdm.rank(feature_reduction_df_16_accuracy.to_numpy(), s_method='TOPSIS', alt_names=['mi_accuracy', 'mrmr_accuracy', 'mi_mrmr_accuracy', 'pca_accuracy']))

print("============================================================ 17 ============================================================")
feature_reduction_df_17_f1 = feature_reduction_df_17[['mi_f1_score', 'mrmr_f1_score', 'mi_mrmr_f1_score', 'pca_f1_score']].reset_index(drop=True).T
feature_reduction_df_17_auc = feature_reduction_df_17[['mi_auc', 'mrmr_auc', 'mi_mrmr_auc', 'pca_auc']].reset_index(drop=True).T
feature_reduction_df_17_accuracy = feature_reduction_df_17[['mi_accuracy', 'mrmr_accuracy', 'mi_mrmr_accuracy', 'pca_accuracy']].reset_index(drop=True).T
print(mcdm.rank(feature_reduction_df_17_f1.to_numpy(), s_method='TOPSIS', alt_names=['mi_f1_score', 'mrmr_f1_score', 'mi_mrmr_f1_score', 'pca_f1_score']))
print(mcdm.rank(feature_reduction_df_17_auc.to_numpy(), s_method='TOPSIS', alt_names=['mi_auc', 'mrmr_auc', 'mi_mrmr_auc', 'pca_auc']))
print(mcdm.rank(feature_reduction_df_17_accuracy.to_numpy(), s_method='TOPSIS', alt_names=['mi_accuracy', 'mrmr_accuracy', 'mi_mrmr_accuracy', 'pca_accuracy']))

============================================================ 8= ===========================================================
[('pca_f1_score', 0.7911081501045316), ('mi_f1_score', 0.29461591182037317), ('mrmr_f1_score', 0.23603003166897715), ('mi_mrmr_f1_score', 0.20108166413309744)]
[('mi_auc', 0.8677006641662602), ('mrmr_auc', 0.7268634557123935), ('mi_mrmr_auc', 0.6074493353791115), ('pca_auc', 0.22209597699953817)]
[('mi_accuracy', 0.879898740940194), ('mrmr_accuracy', 0.8071758571719463), ('mi_mrmr_accuracy', 0.5758207580636645), ('pca_accuracy', 0.21177447779473876)]
============================================================ 9= ===========================================================
[('mi_f1_score', 0.596274045565255), ('pca_f1_score', 0.5384466290923544), ('mi_mrmr_f1_score', 0.5120449328799612), ('mrmr_f1_score', 0.44066401565338287)]
[('mrmr_auc', 0.830529443284165), ('mi_auc', 0.8226647972047744), ('mi_mrmr_auc', 0.6894492762657166), ('pca_auc', 0.09420808853068481)]
[('